In [1]:
import os
from requests import Session

In [2]:
BASE_API_URL = "https://numerbay.ai/backend-api/v1"

NUMERBAY_USERNAME = "YOUR_USERNAME"
NUMERBAY_PASSWORD = "YOUR_PASSWORD"

PRODUCT_ID = 22  # change to your product ID
PRODUCT_ARTIFACT_FILE = "/app/notebooks/research/numerai/example_predictions.csv"  # change to your file path

In [3]:
client = Session()

# Login to get authentication token

In [4]:
login_data = {
    "username": NUMERBAY_USERNAME,
    "password": NUMERBAY_PASSWORD
}

r = client.post(f"{BASE_API_URL}/login/access-token", data=login_data)

In [5]:
access_token = r.json()['access_token']

In [7]:
headers = {"Authorization": f"Bearer {access_token}"}

# List product artifacts

In [8]:
r = client.get(
    f"{BASE_API_URL}/products/{PRODUCT_ID}/artifacts",
    headers=headers,
)
content = r.json()

In [9]:
content

{'total': 0, 'data': []}

# Upload product artifact

In [10]:
artifact_data = {
    "filename": os.path.basename(PRODUCT_ARTIFACT_FILE)
}

r = client.post(
        f"{BASE_API_URL}/products/{PRODUCT_ID}/artifacts/generate-upload-url",
        headers=headers,
        data=artifact_data,
    )

content = r.json()
artifact_id = content['id']
upload_url = content['url']

In [11]:
r = client.put(upload_url, data=open(PRODUCT_ARTIFACT_FILE, 'rb'), headers={"Content-type": "application/octet-stream"})

In [12]:
r.status_code

200

In [13]:
r = client.get(
    f"{BASE_API_URL}/products/{PRODUCT_ID}/artifacts",
    headers=headers,
)
content = r.json()

In [14]:
content

{'total': 1,
 'data': [{'date': '2021-09-19T12:10:36.482649',
   'round_tournament': 281,
   'description': None,
   'url': None,
   'object_name': 'numerai-predictions-restrading_281_5d14b7e8d56d4afb945c7e9f966c5ff9.csv',
   'object_size': None,
   'id': 248,
   'product_id': 22}]}